3 Labels I, O, B (for CPs)

In [ ]:
pip install NERDA

     |████████████████████████████████| 4.0 MB 26.1 MB/s 
     |████████████████████████████████| 6.6 MB 35.4 MB/s 
     |████████████████████████████████| 77 kB 7.2 MB/s 
     |████████████████████████████████| 596 kB 63.8 MB/s 
     |████████████████████████████████| 895 kB 59.2 MB/s 
  Created wheel for progressbar: filename=progressbar-2.5-py3-none-any.whl size=12082 sha256=c8da53cadda28af3e1a56deaad5bcf4d984e13e4b4a5a62c09a197edf5e9c5af
  Stored in directory: /root/.cache/pip/wheels/f0/fd/1f/3e35ed57e94cd8ced38dd46771f1f0f94f65fec548659ed855
Successfully built progressbar
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from collections import defaultdict
import itertools
# import WhitespaceTokenizer() method from nltk
from nltk.tokenize import WhitespaceTokenizer
# Create a reference variable for Class WhitespaceTokenizer
tk = WhitespaceTokenizer()


In [ ]:
# data annotation
def nerda_format(data):
  ner = defaultdict(list)
  for line1,line2 in itertools.zip_longest(*[data]*2):
    sents = []
    tags = []
    tokens = iter(tk.tokenize(line1))
    current_tag = "O"
    for token in tokens:
      try:
        if token == "[":
          current_tag = "I"
          tags.append("B")
          sents.append(next(tokens))
        elif token == "]":
          current_tag = "O"
        else:
          tags.append(current_tag)
          sents.append(token)
      except StopIteration:
        pass
    ner['sentences'].append(sents)
    ner['tags'].append(tags)
  return ner

In [ ]:
#converting annotated data to np.arrary
import numpy as np
from sklearn.model_selection import KFold
from collections import defaultdict

with open('all.txt') as data:
  all = nerda_format(data)

with open('dev.txt') as data:
  dev = nerda_format(data)


all_flat = list(zip(all['sentences'], all['tags']))

#print(all_flat)

all_flat = np.array(all_flat, dtype=object)
print(all_flat)
#print(dev_flat)
print(all_flat.shape)




In [ ]:
#convert np.array to defaultdict
def array2dict(arr):
  dict = defaultdict(list)
  for row in arr:
    tok, lab = tuple(list(col) for col in row)
    dict['sentences'].append(tok)
    dict['tags'].append(lab)
  return dict

In [ ]:
#scheme
from NERDA.models import NERDA

tag_scheme = [
'I',
'B'
]


In [43]:
#setup number of folds(splits)
kf = KFold(n_splits=10)
split = 1
for train_index, test_index in kf.split(all_flat):
  X_train, X_test = all_flat[train_index], all_flat[test_index]
  training = array2dict(X_train)
  testing = array2dict(X_test)
  
  model = NERDA(
  dataset_training = training,
  dataset_validation = dev,
  tag_scheme = tag_scheme, 
  tag_outside = 'O',
  transformer = 'bert-base-multilingual-cased',
  hyperparameters = {'epochs' : 11,
                  'warmup_steps' : 10,
                  'train_batch_size': 7,
                  'learning_rate': 0.0001})
  model.train()
  df = model.evaluate_performance(testing)
  df.to_csv('out_%s.csv' % str(split), encoding='utf-8', index=False)
  #print("Split %d: " % split, "TRAIN:", training, "\tTEST:", testing)
  split += 1



Device automatically set to: cuda


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be r


 Epoch 1 / 11


  9%|▉         | 2/22 [00:00<00:05,  3.48it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #31 length 150 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #32 length 130 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #33 length 174 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
 14%|█▎        | 3/22 [00:00<00:04,  3.96it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #36 length 233 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #37 length 149 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
 23%|██▎       | 5/22 [00:01<00:03,  4.36it/s]/usr/local/lib/python3.7/di

Train Loss = 0.8557198156010021 Valid Loss = 0.9258134166399637

 Epoch 2 / 11


...

Train Loss = 0.2693489571525292 Valid Loss = 0.7757552737991015

 Epoch 10 / 11


  0%|          | 0/22 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #6 length 262 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #14 length 152 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
 18%|█▊        | 4/22 [00:00<00:04,  4.28it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #48 length 150 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
 23%|██▎       | 5/22 [00:01<00:03,  4.34it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #49 length 130 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #50 length 174 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packag

Train Loss = 0.23579998094249854 Valid Loss = 0.6818299094835917

 Epoch 11 / 11


  0%|          | 0/22 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #6 length 262 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #14 length 152 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
 18%|█▊        | 4/22 [00:01<00:04,  4.26it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #48 length 150 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
 23%|██▎       | 5/22 [00:01<00:03,  4.33it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #49 length 130 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #50 length 174 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packag

Train Loss = 0.19480633938854391 Valid Loss = 0.6954169571399689


Device automatically set to: cuda


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be r


 Epoch 1 / 11


  0%|          | 0/22 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #6 length 262 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #14 length 152 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
  9%|▉         | 2/22 [00:00<00:05,  3.45it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #31 length 150 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #32 length 130 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #33 length 174 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
 14%|█▎        | 3/22 [00:00<00:04,  3.94it/s]/usr/local/lib/python3.7/dist-packag

Train Loss = 0.8269074654037302 Valid Loss = 0.8953990439573923

 Epoch 2 / 11


...

  0%|          | 0/22 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #6 length 262 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #14 length 152 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
  9%|▉         | 2/22 [00:00<00:05,  3.69it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #31 length 150 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #32 length 130 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #33 length 174 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
 14%|█▎        | 3/22 [00:00<00:04,  4.06it/s]/usr/local/lib/python3.7/dist-packag

Train Loss = 0.04560473461804742 Valid Loss = 1.0744890390584867

 Epoch 11 / 11


  0%|          | 0/22 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #6 length 262 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #14 length 152 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
  9%|▉         | 2/22 [00:00<00:05,  3.82it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #31 length 150 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #32 length 130 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #33 length 174 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
 14%|█▎        | 3/22 [00:00<00:04,  4.10it/s]/usr/local/lib/python3.7/dist-packag

Train Loss = 0.03945068291134455 Valid Loss = 1.0790673283239205


       Level  F1-Score  Precision  Recall
0          I  0.948454   0.901961  1.0000
1          B  0.787879   0.764706  0.8125
0  AVG_MICRO  0.935867        NaN     NaN
0  AVG_MICRO  0.868166        NaN     NaN


In [46]:
import os
import glob
import pandas as pd

extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])

combined_csv.to_csv( "10folds.csv", index=False, encoding='utf-8-sig')